In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
import sklearn
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score

In [3]:
from scipy.stats import pearsonr

In [4]:
%store -r df_Xy

In [5]:
df_Xy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3032 entries, 2008-09-05 to 2020-05-27
Columns: 242 entries, gbp_rate to gold_change%_lag10
dtypes: float64(138), object(104)
memory usage: 5.6+ MB


In [7]:
df_X = df_Xy.drop(columns = ['gold','gold_change%'])

In [8]:
df_X.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3032 entries, 2008-09-05 to 2020-05-27
Columns: 240 entries, gbp_rate to gold_change%_lag10
dtypes: float64(136), object(104)
memory usage: 5.6+ MB


In [9]:
df_y = df_Xy['gold']

In [10]:
model = GradientBoostingRegressor()
cross_val_score(model,df_X,df_y)

ValueError: could not convert string to float: '~'